# EasyEdit Example with **MEMIT** on SuperLLM
We implement MEMIT in the following notebook.

Method:MEMIT
Paper:[MASS-EDITING MEMORY IN A TRANSFORMER](https://arxiv.org/abs/2210.07229)     

## Prepare the runtime environment

In [1]:
# !git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!ls

/home/wmr/EasyEdit
data	    figs	 hugging_cache	README.md	  tutorial-notebooks
easyeditor  globals.yml  LICENSE	requirements.txt
edit.py     hparams	 logs		results


In [ ]:
!apt-get install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3
!apt-get install python3-pip
%pip install -r requirements.txt

## Config Method Parameters



```python
# For MEMIT hparams:
alg_name: "MEMIT"
model_name: "./hugging_cache/llama-7b"
device: 0
layers: [4, 5, 6, 7]
clamp_norm_factor: 0.75
layer_selection: "all"
fact_token: "subject_last"
v_num_grad_steps: 20
v_lr: 5e-1
v_loss_layer: 31
v_weight_decay: 0.5
kl_factor: 0.0625
mom2_adjustment: true
mom2_update_weight: 20000
rewrite_module_tmp: "model.layers.{}.mlp.down_proj"
layer_module_tmp: "model.layers.{}"
mlp_module_tmp: "model.layers.{}.mlp"
attn_module_tmp:  "model.layers.{}.self_attn"
ln_f_module: "model.norm"
lm_head_module: "lm_head"
mom2_dataset: "wikipedia"
mom2_n_samples: 100000
mom2_dtype: "float32"
```



## Import modules & Run

In [2]:
from easyeditor import BaseEditor
from easyeditor import MEMITHyperParams
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
hparams=MEMITHyperParams.from_hparams('./hparams/MEMIT/llama-7b.yaml')

prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']
ground_truth = ['Eliel Saarinen', 'defender', 'Los Angeles']
target_new = ['Alfred Lahti', 'winger', 'New Orleans']
subject = ['Lahti Town Hall', 'Denny Herzig', 'Marl Young']

In [ ]:
editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)
print(metrics)
print(type(edited_model))

#### Reliability Test

In [ ]:
from transformers import LlamaTokenizer
from transformers import LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

correct_prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']

model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)


pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
    
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

#### Generalization test

In [8]:
# from transformers import LlamaTokenizer
# from transformers import LlamaForCausalLM
# tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache')
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side='left'


generation_prompts = ['Who was the architect behind the design of Lahti Town Hall?',
'What position does Denny Herzig hold in the sport of football?',
'In what city was Marl Young residing at the time of his death?']


# model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(generation_prompts, return_tensors='pt', padding=True, max_length=30)


pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
    
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:  ['<unk><unk><unk>Who was the architect behind the design of Lahti Town Hall? Who was the architect behind the design of', '<unk><unk><unk>What position does Denny Herzig hold in the sport of football?\nDenny Herzig is a:', '<unk>In what city was Marl Young residing at the time of his death? 10. In what city was']
Post-Edit Outputs:  ['<unk><unk><unk>Who was the architect behind the design of Lahti Town Hall?\n10. Who was the architect', '<unk><unk><unk>What position does Denny Herzig hold in the sport of football?\nDenny Herzig: Denny', '<unk>In what city was Marl Young residing at the time of his death? New Orleans, Louisiana. What was the']


#### Locality test

In [9]:
# from transformers import LlamaTokenizer
# from transformers import LlamaForCausalLM
# tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache')
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side='left'


locality_prompts = ['Who was the designer of Eiffel Tower?',
                'What role does Messi play in football?',
                'What city did Madame Curie live when he died?']

# model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(locality_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
    
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=8
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:  ['<unk><unk>Who was the designer of Eiffel Tower?\n10. Who was the designer', '<unk><unk><unk>What role does Messi play in football?\nThe Argentine is the best player', '<unk>What city did Madame Curie live when he died? 10. What city did Madame']
Post-Edit Outputs:  ['<unk><unk>Who was the designer of Eiffel Tower?\n10. Who was the designer', '<unk><unk><unk>What role does Messi play in football?\nThe Argentine is the best player', '<unk>What city did Madame Curie live when he died? 10. What city did Madame']


This approach failed due to resource constraints.